In [1]:
import warnings
warnings.filterwarnings("ignore")
import torch
import sys
import os
import numpy as np
from tqdm import tqdm
import glob

import torchaudio
from speechbrain.pretrained.interfaces import foreign_class

torchvision is not available - cannot save figures


In [2]:
classifier = foreign_class(source="/mntcephfs/lab_data/shoinoue/Models/trained_models/accent/CustomEncoderWav2vec2Classifier-a72df039c801fa14a1c3226e95ab8c14/", pymodule_file="custom_interface.py", classname="CustomEncoderWav2vec2Classifier", savedir="/mntcephfs/lab_data/shoinoue/Models/trained_models/accent/CustomEncoderWav2vec2Classifier-a72df039c801fa14a1c3226e95ab8c14/", run_opts={"device":"cuda"})

speechbrain.lobes.models.huggingface_wav2vec - wav2vec 2.0 is frozen.


In [5]:
data_dir = "/mntcephfs/lab_data/shoinoue/"
dataset_dir = data_dir + "Dataset/CMU-ARCTIC/"
fs = 16000
speakers = [os.path.basename(a) for a in glob.glob(dataset_dir + "*")]
speakers.sort()

In [8]:
save = True

for spk in speakers:
    print(spk)
    ae_dir = f"{dataset_dir}{spk}/accent_embedding/"
    os.makedirs(ae_dir, exist_ok=True)
    filenames = glob.glob(dataset_dir + spk + "/wav/*")
    filenames.sort()
    for path in tqdm(filenames):
        signal, org_sr = torchaudio.load(path)
        signal = torchaudio.functional.resample(signal, orig_freq=org_sr, new_freq=fs)
        embeddings =  classifier.encode_batch(signal)[0].detach().cpu().numpy()
        savepath = ae_dir + os.path.basename(path)[:-4] + ".npy"
        if save:
            np.save(savepath, embeddings)

SLT


100%|██████████| 1132/1132 [00:45<00:00, 24.84it/s]
